In [10]:
import ee
import geemap
import collections
collections.Callable = collections.abc.Callable
ee.Initialize()

In [7]:
Map = geemap.Map()

# On 21st February 2019, massive forest fires broke out in
# numerous places across the Bandipur National Park of
# the Karnataka state in India.
# By 25 February 2019 most fire was brought under control
# This script shows how to do damage assessment using
# spectral index change detection technique.

# Define the area of interest
geometry = ee.Geometry.Polygon([[
  [76.37639666685044, 11.766523239445169],
  [76.37639666685044, 11.519036946599561],
  [76.78426409849106, 11.519036946599561],
  [76.78426409849106, 11.766523239445169]
]])
fireStart = ee.Date('2019-02-20')
fireEnd = ee.Date('2019-02-25')

Map.centerObject(geometry, 10)

s2 = ee.ImageCollection("COPERNICUS/S2")

# Write a function for Cloud masking
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
        qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask) \
    .select("B.*") \
    .copyProperties(image, ["system:time_start"])

# Apply filters and cloud mask
filtered = s2 \
    .filter(ee.Filter.bounds(geometry)) \
    .map(maskS2clouds) \
    .select('B.*')

# Create Before and After composites
before = filtered \
    .filter(ee.Filter.date(
        fireStart.advance(-2, 'month'), fireStart)) \
    .median()

after = filtered \
  .filter(ee.Filter.date(
    fireEnd, fireEnd.advance(1, 'month'))) \
  .median()

# Freshly burnt regions appeat bright in SWIR-bands
# Use a False Color Visualization
swirVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B12', 'B8', 'B4'],
}
Map.addLayer(before, swirVis, 'Before')
Map.addLayer(after, swirVis, 'After')

# Write a function to calculate  Normalized Burn Ratio (NBR)
# 'NIR' (B8) and 'SWIR-2' (B12)
def addNBR(image):
    ndbi = image.normalizedDifference(['B8', 'B12']).rename(['nbr'])
    return image.addBands(ndbi)

beforeNbr = addNBR(before).select('nbr')
afterNbr = addNBR(after).select('nbr')

nbrVis = {'min': -0.5, 'max': 0.5, 'palette': ['white', 'black']}

Map.addLayer(beforeNbr, nbrVis, 'Prefire NBR')
Map.addLayer(afterNbr, nbrVis, 'Postfire NBR')

# Calculate Change in NBR (dNBR)
change = beforeNbr.subtract(afterNbr)

# Apply a threshold
threshold = 0.3

# Display Burned Areas
burned = change.gt(threshold)
Map.addLayer(burned, {'min':0, 'max':1, 'palette': ['white', 'red']}, 'Burned', False)
Map

Map(center=[11.642833473159406, 76.58033038267084], controls=(WidgetControl(options=['position', 'transparent_…

In [8]:
Map = geemap.Map()

# Define the area of interest
geometry = ee.Geometry.Polygon([[
  [76.37639666685044, 11.766523239445169],
  [76.37639666685044, 11.519036946599561],
  [76.78426409849106, 11.519036946599561],
  [76.78426409849106, 11.766523239445169]
]])
fireStart = ee.Date('2019-02-20')
fireEnd = ee.Date('2019-02-25')

Map.centerObject(geometry, 10)

s2 = ee.ImageCollection("COPERNICUS/S2")

# Write a function for Cloud masking
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask) \
    .select("B.*") \
    .copyProperties(image, ["system:time_start"])

# Apply filters and cloud mask
filtered = s2 \
  .filter(ee.Filter.bounds(geometry)) \
  .map(maskS2clouds) \
  .select('B.*')

# Create Before and After composites
before = filtered \
  .filter(ee.Filter.date(
    fireStart.advance(-2, 'month'), fireStart)) \
  .median()

after = filtered \
  .filter(ee.Filter.date(
    fireEnd, fireEnd.advance(1, 'month'))) \
  .median()

# Write a function to calculate  Normalized Burn Ratio (NBR)
# 'NIR' (B8) and 'SWIR-2' (B12)
def addNBR(image):
    ndbi = image.normalizedDifference(['B8', 'B12']).rename(['nbr'])
    return image.addBands(ndbi)

beforeNbr = addNBR(before).select('nbr')
afterNbr = addNBR(after).select('nbr')

# Calculate Change in NBR (dNBR)
change = beforeNbr.subtract(afterNbr)

dnbrPalette = ['#ffffb2','#fecc5c','#fd8d3c','#f03b20','#bd0026']
# Display the change image
Map.addLayer(change, {'min':0.1, 'max': 0.7, 'palette': dnbrPalette},
  'Change in NBR')

# We can also classify the change image according to
# burn severity

# United States Geological Survey (USGS) proposed
# a classification table to interpret the burn severity
# We will assign a discrete class value and visualize it
# | Severity     | dNBR Range         | Class |
# |--------------|--------------------|-------|
# | Unburned     | < 0.1              | 0     |
# | Low Severity | >= 0.10 and <0.27  | 1     |
# | Moderate-Low | >= 0.27 and <0.44  | 2     |
# | Moderate-High| >= 0.44 and< 0.66  | 3     |
# | High         | >= 0.66            | 4     |

# Classification of continuous values can be done
# using the .where() function
severity = change \
  .where(change.lt(0.10), 0) \
  .where(change.gte(0.10).And(change.lt(0.27)), 1) \
  .where(change.gte(0.27).And(change.lt(0.44)), 2) \
  .where(change.gte(0.44).And(change.lt(0.66)), 3) \
  .where(change.gt(0.66), 4)

# Exercise

# The resulting image 'severiy' is a discrete image with
# pixel values from 0-4 representing the severity class

# Display the image according to the following color table

# | Severity     | Class | Color   |
# |--------------|-------|---------|
# | Unburned     | 0     | green   |
# | Low Severity | 1     | yellow  |
# | Moderate-Low | 2     | organge |
# | Moderate-High| 3     | red     |
# | High         | 4     | magenta |

Map.addLayer(severity, {'min': 0, 'max': 4, 'palette': ['green', 'yellow', 'orange', 'red', 'magenta']}, 'severity')

Map




Map(center=[11.642833473159406, 76.58033038267084], controls=(WidgetControl(options=['position', 'transparent_…

In [13]:
Map = geemap.Map()

geometry = ee.Geometry.Polygon([[
  [75.70357667713435, 12.49723970868507],
  [75.70357667713435, 12.470171844429931],
  [75.7528434923199, 12.470171844429931],
  [75.7528434923199, 12.49723970868507]
]])
Map.centerObject(geometry)
s2 = ee.ImageCollection("COPERNICUS/S2")
rgbVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
}

# Write a function for Cloud masking
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask) \
      .select("B.*") \
      .copyProperties(image, ["system:time_start"])

filtered = s2 \
  .filter(ee.Filter.bounds(geometry)) \
  .map(maskS2clouds) \
  .select('B.*')

dateOfIncident = ee.Date('2018-12-15')
before = filtered \
  .filter(ee.Filter.date(dateOfIncident.advance(-2, 'year'), dateOfIncident)) \
  .filter(ee.Filter.calendarRange(12, 12, 'month')) \
  .median()
after = filtered.filter(ee.Filter.date(
  dateOfIncident, dateOfIncident.advance(1, 'month'))).median()

Map.addLayer(before, rgbVis, 'Before')
Map.addLayer(after, rgbVis, 'After')

# Calculate Distance
# Formula at https:#www.varsitytutors.com/calculus_3-help/distance-between-vectors
def magnitude(image):
    return image.pow(2).reduce(ee.Reducer.sum()).sqrt()

distance = magnitude(after.subtract(before))

# Calculate Angle
# https:#byjus.com/angle-between-two-vectors-formula/
dot = before.multiply(after).reduce(ee.Reducer.sum())

angle = dot.divide(magnitude(after)) \
              .divide(magnitude(before)) \
              .acos()

Map.addLayer(distance, {'min': 0, 'max': 1500, 'palette': ['white', 'red']}, 'spectral distance')
Map.addLayer(angle, {'min': 0, 'max': 1, 'palette': ['white', 'purple']}, 'angle')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [15]:
Map = geemap.Map()

geometry = ee.Geometry.Polygon([[
  [75.70357667713435, 12.49723970868507],
  [75.70357667713435, 12.470171844429931],
  [75.7528434923199, 12.470171844429931],
  [75.7528434923199, 12.49723970868507]
]])
Map.centerObject(geometry)
s2 = ee.ImageCollection("COPERNICUS/S2")
rgbVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
}

# Write a function for Cloud masking
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask) \
      .select("B.*") \
      .copyProperties(image, ["system:time_start"])

filtered = s2 \
  .filter(ee.Filter.bounds(geometry)) \
  .map(maskS2clouds) \
  .select('B.*')

dateOfIncident = ee.Date('2018-12-15')
before = filtered \
  .filter(ee.Filter.date(dateOfIncident.advance(-2, 'year'), dateOfIncident)) \
  .filter(ee.Filter.calendarRange(12, 12, 'month')) \
  .median()
after = filtered.filter(ee.Filter.date(
  dateOfIncident, dateOfIncident.advance(1, 'month'))).median()

Map.addLayer(before, rgbVis, 'Before')
Map.addLayer(after, rgbVis, 'After')

def magnitude(image):
    return image.pow(2).reduce(ee.Reducer.sum()).sqrt()

distance = magnitude(after.subtract(before))

Map.addLayer(distance, {'min': 0, 'max': 1500, 'palette': ['white', 'red']}, 'spectral distance')

# Exercise
# Inspect the distance image and find a suitable threshold
# that signifies damage after the landslides
# Apply the threshold and create a new image showing landslides
# Display the results

# Hint: Use the .gt() method to apply the threshold
dmg = distance \
  .where(distance.lt(1200.0), 0) \
  .where(distance.gt(1201.0), 1)

Map.addLayer(dmg, {'min': 0, 'max': 1, 'palette': ['white', 'red']}, 'landslides')

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [24]:
Map = geemap.Map()

bangalore = ee.FeatureCollection('users/ujavalgandhi/public/bangalore_boundary')
change = ee.FeatureCollection('users/ujavalgandhi/e2e/bangalore_change_gcps')
nochange = ee.FeatureCollection('users/ujavalgandhi/e2e/bangalore_nochange_gcps')
s2 = ee.ImageCollection("COPERNICUS/S2")

rgbVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
}

# Write a function for Cloud masking
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000) \
      .select("B.*") \
      .copyProperties(image, ["system:time_start"])

filtered = s2 \
  .filter(ee.Filter.date('2019-01-01', '2019-02-01')) \
  .filter(ee.Filter.bounds(bangalore)) \
  .map(maskS2clouds)

image2019 = filtered.median().clip(bangalore)
# Display the input composite.
Map.addLayer(image2019, rgbVis, '2019')

filtered = s2 \
  .filter(ee.Filter.date('2020-01-01', '2020-02-01')) \
  .filter(ee.Filter.bounds(bangalore)) \
  .map(maskS2clouds)

image2020 = filtered.median().clip(bangalore)

Map.addLayer(image2020, rgbVis, '2020')

stackedImage = image2019.addBands(image2020)

merged = change.merge(nochange)

# Overlay the point on the image to get training data.
training = stackedImage.sampleRegions(
  collection=merged, properties=['class'], scale=10)

# Train a classifier.
classifier = ee.Classifier.smileRandomForest(50).train(
  features=training, classProperty='class', inputProperties=stackedImage.bandNames())

# Classify the image.
classified = stackedImage.classify(classifier)
Map.addLayer(classified, {'min': 0, 'max': 1, 'palette': ['white', 'red']}, 'change')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [27]:
Map = geemap.Map()

bangalore = ee.FeatureCollection('users/ujavalgandhi/public/bangalore_boundary')
change = ee.FeatureCollection('users/ujavalgandhi/e2e/bangalore_change_gcps')
nochange = ee.FeatureCollection('users/ujavalgandhi/e2e/bangalore_nochange_gcps')
s2 = ee.ImageCollection("COPERNICUS/S2")

Map.centerObject(bangalore)

rgbVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
}

# Write a function for Cloud masking
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask) \
      .select("B.*") \
      .copyProperties(image, ["system:time_start"])

filtered = s2 \
  .filter(ee.Filter.date('2019-01-01', '2019-02-01')) \
  .filter(ee.Filter.bounds(bangalore)) \
  .map(maskS2clouds)

image2019 = filtered.median().clip(bangalore)
# Display the input composite.
Map.addLayer(image2019, rgbVis, '2019')

filtered = s2 \
  .filter(ee.Filter.date('2020-01-01', '2020-02-01')) \
  .filter(ee.Filter.bounds(bangalore)) \
  .map(maskS2clouds)

image2020 = filtered.median().clip(bangalore)

Map.addLayer(image2020, rgbVis, '2020')

# Exercise

# Let's add an NDBI band that can improve the detection
def addNDBI(image):
    ndbi = image.normalizedDifference(['B11', 'B8']).rename(['ndbi'])
    return image.addBands(ndbi)

# use addNDBI() function to add the NDBI band to both 2019 and 2020 composite images
# Hint1: You can save the resulting image in the same variable to avoid changing
# a lot of code.
# image = addNDBI(image)

image2019 = addNDBI(image2019)
image2020 = addNDBI(image2020)

stackedImage = image2019.addBands(image2020)

merged = change.merge(nochange)

# Overlay the point on the image to get training data.
training = stackedImage.sampleRegions(
  collection=merged, properties=['class'], scale=10)

# Train a classifier.
classifier = ee.Classifier.smileRandomForest(50).train(
  features=training, classProperty='class', inputProperties=stackedImage.bandNames())

# Classify the image.
classified = stackedImage.classify(classifier)
Map.addLayer(classified, {'min': 0, 'max': 1, 'palette': ['white', 'red']}, 'change')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [29]:
Map = geemap.Map()

bangalore = ee.FeatureCollection("users/ujavalgandhi/public/bangalore_boundary")
urban = ee.FeatureCollection("users/ujavalgandhi/e2e/urban_gcps")
bare = ee.FeatureCollection("users/ujavalgandhi/e2e/bare_gcps")
water = ee.FeatureCollection("users/ujavalgandhi/e2e/water_gcps")
vegetation = ee.FeatureCollection("users/ujavalgandhi/e2e/vegetation_gcps")
s2 = ee.ImageCollection("COPERNICUS/S2_SR")

Map.centerObject(bangalore)
rgbVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
}

filtered = s2 \
  .filter(ee.Filter.date('2019-01-01', '2019-02-01')) \
  .filter(ee.Filter.bounds(bangalore)) \
  .select('B.*')

before = filtered.median().clip(bangalore)
# Display the input composite.
Map.addLayer(before, rgbVis, 'before')

training = urban.merge(bare).merge(water).merge(vegetation)

# Overlay the point on the image to get training data.
training = before.sampleRegions(
  collection=training,
  properties=['landcover'],
  scale=10
)

# Train a classifier.
classifier = ee.Classifier.smileRandomForest(50).train(
  features=training,
  classProperty='landcover',
  inputProperties=before.bandNames()
)

# # Classify the image.
beforeClassified = before.classify(classifier)
Map.addLayer(beforeClassified,
  {'min': 0, 'max': 3, 'palette': ['gray', 'brown', 'blue', 'green']}, 'before_classified')

# 2020 Jan
after = s2 \
  .filter(ee.Filter.date('2020-01-01', '2020-02-01')) \
  .filter(ee.Filter.bounds(bangalore)) \
  .select('B.*') \
  .median() \
  .clip(bangalore)

Map.addLayer(after, rgbVis, 'after')

# Classify the image.
afterClassified= after.classify(classifier)
Map.addLayer(afterClassified,
  {'min': 0, 'max': 3, 'palette': ['gray', 'brown', 'blue', 'green']}, 'after_classified')

# Reclassify from 0-3 to 1-4
beforeClasses = beforeClassified.remap([0, 1, 2, 3], [1, 2, 3, 4])
afterClasses = afterClassified.remap([0, 1, 2, 3], [1, 2, 3, 4])

# Show all changed areas
changed = afterClasses.subtract(beforeClasses).neq(0)
Map.addLayer(changed, {'min':0, 'max':1, 'palette': ['white', 'red']}, 'Change')

# We multiply the before image with 100 and add the after image
# The resulting pixel values will be unique and will represent each unique transition
# i.e. 102 is urban to bare, 103 urban to water etc.
merged = beforeClasses.multiply(100).add(afterClasses).rename('transitions')

# Use a frequencyHistogram to get a pixel count per class
transitionMatrix = merged.reduceRegion(
  reducer=ee.Reducer.frequencyHistogram(),
  geometry=bangalore,
  maxPixels=1e10,
  scale=10,
  tileScale=16
)
# This prints number of pixels for each class transition
print(transitionMatrix.get('transitions').getInfo())

# If we want to calculate the area of each class transition
# we can use a grouped reducer

# Divide by 1e6 to get the area in sq.km.
areaImage = ee.Image.pixelArea().divide(1e6).addBands(merged)
# Calculate Area by each Transition Class
# using a Grouped Reducer
areas = areaImage.reduceRegion(
    reducer=ee.Reducer.sum().group(
        groupField=1,
        groupName='transitions',
    ),
    geometry=bangalore,
    scale=100,
    tileScale=4,
    maxPixels=1e10
    )

# Post-process the result to generate a clean output
classAreas = ee.List(areas.get('groups'))

def func_wlb(item):
    areaDict = ee.Dictionary(item)
    classNumber = ee.Number(areaDict.get('transitions')).format()
    area = ee.Number(areaDict.get('sum')).round()
    return ee.List([classNumber, area])

classAreaLists = classAreas.map(func_wlb)

classTransitionsAreaDict = ee.Dictionary(classAreaLists.flatten())
print(classTransitionsAreaDict.getInfo())
Map

{'101': 4747324.121568627, '102': 170293.53725490186, '103': 13782.603921568627, '104': 201643.80784313733, '201': 188329.3098039216, '202': 418340.1529411765, '203': 1624.4235294117648, '204': 8723.254901960783, '301': 9144.388235294118, '302': 597, '303': 69543.36470588236, '304': 19134.705882352944, '401': 300955.49019607855, '402': 23805.79607843137, '403': 5070.105882352941, '404': 1159799.1411764706}
{'101': 526, '102': 14, '103': 1, '104': 9, '201': 13, '202': 29, '203': 0, '204': 0, '301': 1, '302': 0, '303': 5, '304': 1, '401': 26, '402': 1, '403': 0, '404': 84}


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [38]:
Map = geemap.Map()

# Exercise
# Show all areas where water became other classes and display the result
# Hint1: Select class 3 pixels from before image and NOT class 3 pixels from after image
# Hint2: use the .And() operation to select pixels matching both conditions
Map.centerObject(bangalore)

nowater = beforeClasses.eq(3).And(afterClasses.neq(3))

Map.addLayer(nowater, {'min':0, 'max':1, 'palette': ['white', 'red']}, 'Water Changed')

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…